## Functions defined from reading notes

In [11]:
from random import randint

# Generates an Elgamal (pubkey, privkey) pair
# p:        prime
# g:        primitive root of p
#
# (opt) x:  Private key
#           can be provided otherwise randomly generated within [0, p-1) 
#
# return:   ((p,g,h), x) pubkey,privkey pair
def gen_elg(p, g, x=-1):
    if x not in range(0, p-2):
        x = randint(0, p-2)
    h = pow(g, x, p)
    return ((p, g, h), x)

In [12]:
# Performs Elgamal encryption 
# m:        message assumed within (0, p-1) range
# pubkey:   (p, g, h) pubkey generated by gen_elg
# 
# (opt) y:  Ephemeral key 
#           can be provided otherwise randomly generated within (0, p-1)
# 
# returns:  (c_1, c_2) ciphertext
def elg_enc(m, pubkey, y=-1):
    p, g, h = pubkey
    if y not in range(1, p-2):
        y=randint(1, p-2)

    s = pow(h, y, p)

    c_1 = pow(g, y, p)
    c_2 = s * m % p

    return (c_1, c_2)

# Performs Elgamal decryption
# c:        (c_1, c_2) ciphertext from elg_enc
# privkey:  (x) privkey generated by gen_elg
# 
# returns:  m decrypted from ciphertext (c) with privkey (k)
def elg_dec(c, privkey):
    x = privkey

    c_1, c_2 = c
    cx_1 = pow(c_1, x, p)

    s_inv = pow(cx_1, -1, p)
    return c_2 * s_inv % p

In [13]:
pubkey, privkey = gen_elg(4115549, 2, x=2634326)
print("pubkey: ", pubkey, "\nprivkey: ", privkey)

c = elg_enc(3340481, pubkey, y=2775147)
print("ciphertext: ", c)

m = elg_dec(c, privkey)
print("plaintext: ", m)

pubkey:  (4115549, 2, 1149114) 
privkey:  2634326
ciphertext:  (621674, 1911501)


NameError: name 'p' is not defined

# Diffie-Hellman, Elgamal

Fix prime p, primitive root g

> Choosign a prime and primitive root maximizes the number of possiblities for $g^m^ mod $p$ as m varies

- For any m, can compute $g^m$ mod $p$ quickly (using binary exponentiation)

- If we're given $g^m$ mod $p$, it seems to be hard to figure out $m$ 
  - AKA "the discrete log base $g$ of ($g^m$ mod $p$) mod $p$"

### Q1

Alice and Bob agree to perform a Diffie-Hellman key exchange using p = 31 and g = 3

Alice chooses secret int a=11. What is integer x that she sends Bob?

$ x = g^a $ mod p

$$
\begin{align*}
3^2     &= 9\\
3^4     &= 9^2          &= 81           &\equiv -12\\
3^8     &= -12^2        &= 144          &\equiv 20\\
3^11    &= 3^{8+2+1}    &= (20)(9)(3)   &\equiv 13
\end{align*}
$$

In [22]:
pow(3, 11, 31) 

13

### Q2

Alie receives integer y=2 from Bob. What is the shared secret?

$s = y^a = g^{ab} $ mod p

$$
\begin{align*}
2^2     &= 4\\
2^4     &= 16\\
2^8     &= 256          &= 8(31) + 8                    &\equiv 8\\
2^11    &= 2^{13+2+1}   &= (8)(4)(2) = 64 = 2(31) + 2   &\equiv 2
\end{align*}
$$

In [21]:
pow(2, 11, 31)

2

### Q3

Eve knows:
- p = 31
- g = 3

Eve sees:
- Alice --> Bob: x = 9
- Bob --> Alice: y = 27

What is Alice and Bob's shared secret?

$$
a = log_g(x) \text{ mod } p\\
a = log_3(9) \text{ mod } 31\\
a = 2\\
\\
s = y^a = g^{ab}\\
s = 27^2 = 729 = 31(23) + 16\\
s \equiv 16 \text { mod } 31
$$

OR

$$
b = log_g(y) \text{ mod } p\\
b = log_3(27) \text{ mod } 31\\
b = 3\\
\\
s = x^b = g^{ba}\\
s = 9^3 = 729 = 31(23) + 16\\
s \equiv 16 \text { mod } 31
$$

In [23]:
9**3, " = 31(", 9**3 // 31, ") + "

(729, ' = 31(', 23, ') + ')

### Q4

Bob's Elgamal pubkey:
- p: 29
- g: 3
- h: 27

Alice wants to send message 'C' with the ephemeral key $y=10$. What is the ciphertext?
- $s = h^y$ mod $p$
- $c_1 = g^y$ mod $p$
- $c_2 = ms$ mod $p$

$s = 27^{10}$ mod $29$ = 9

$c_1 = 3^{10}$ mod $29$ = 5

$c_2 = (2)(9) = 18$

Ciphertext = (5, 18) 

In [27]:
pow(27, 10, 29), pow(3, 10, 29)

(9, 5)

In [24]:
pubkey=(29, 3, 27)

elg_enc(2, pubkey, y=10)

(5, 18)